In [1]:
import pandas as pd
import numpy as np
import glob
import os
import random
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import datetime

In [2]:
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, LSTM, Flatten, BatchNormalization, Dropout
from keras.utils import to_categorical
import tensorflow as tf
# Load the TensorBoard notebook extension
%load_ext tensorboard

Using TensorFlow backend.


In [3]:
enc = pickle.load( open( "pickle\\enc.p", "rb" ) )
val_x = pickle.load( open( "pickle\\val_x.p", "rb" ) )
train_x = pickle.load( open( "pickle\\train_x.p", "rb" ) )
val_labels = pickle.load( open( "pickle\\val_labels.p", "rb" ) )
train_labels = pickle.load( open( "pickle\\train_labels.p", "rb" ) )
class_weights = pickle.load( open( "pickle\\class_weights.p", "rb" ) )
class_weights

array([2.13490121, 1.14730664, 2.29595517, 0.76382944, 1.31395025,
       1.07741035, 1.34639346, 2.89534169, 0.57541893, 1.00927592,
       0.72144126, 0.49428218])

In [4]:
log_dir = os.path.join(
    "Singlelogs",
    "fit",
    "1632",
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),
)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [5]:
max(np.sum(val_labels, axis = 0))/sum(np.sum(val_labels, axis = 0))

0.16050966307508124

In [6]:
model = keras.Sequential()
model.add(LSTM(16, 
               input_shape=(train_x.shape[1], train_x.shape[2]), 
               return_sequences = True))
model.add(LSTM(32, 
               input_shape=(train_x.shape[1], train_x.shape[2]), 
               return_sequences = False))
model.add(Dense(200, activation= "softmax"))
#model.add(BatchNormalization())
#model.add(Dropout(.25))
model.add(Dense(len(enc.categories_[0]), activation= "softmax"))
adam = keras.optimizers.Adam(lr=0.0001)
model.compile(loss = "categorical_crossentropy", optimizer= adam, metrics=["acc"])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 200, 16)           2816      
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                6272      
_________________________________________________________________
dense (Dense)                (None, 200)               6600      
_________________________________________________________________
dense_1 (Dense)              (None, 12)                2412      
Total params: 18,100
Trainable params: 18,100
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(train_x, train_labels,
          epochs = 50, 
          shuffle = True,
          validation_data = (val_x, val_labels),
          batch_size = 2048,
         class_weight = class_weights,
         callbacks=[tensorboard_callback])

Train on 47113 samples, validate on 11694 samples
Epoch 1/50
47113/47113 [==============================] - 36s 767us/sample - loss: 2.4849 - acc: 0.0789 - val_loss: 2.4838 - val_acc: 0.1117
Epoch 2/50
47113/47113 [==============================] - 38s 814us/sample - loss: 2.4829 - acc: 0.1155 - val_loss: 2.4819 - val_acc: 0.1117
Epoch 3/50
47113/47113 [==============================] - 43s 909us/sample - loss: 2.4809 - acc: 0.1155 - val_loss: 2.4800 - val_acc: 0.1117
Epoch 4/50
47113/47113 [==============================] - 43s 913us/sample - loss: 2.4789 - acc: 0.1155 - val_loss: 2.4780 - val_acc: 0.1117
Epoch 5/50
47113/47113 [==============================] - 44s 933us/sample - loss: 2.4768 - acc: 0.1155 - val_loss: 2.4759 - val_acc: 0.1117
Epoch 6/50
47113/47113 [==============================] - 44s 927us/sample - loss: 2.4745 - acc: 0.1155 - val_loss: 2.4736 - val_acc: 0.1117
Epoch 7/50
47113/47113 [==============================] - 44s 925us/sample - loss: 2.4719 - acc: 0.1155 

In [ ]:
preds = model.predict(train_x, batch_size = 256, verbose = 1)
print(np.sum(preds, axis = 0))
print(np.sum(train_labels, axis = 0))
plt.imshow(
    confusion_matrix(
        enc.inverse_transform(preds), 
        enc.inverse_transform(train_labels), 
        normalize = "true"
    )
)

In [ ]:
preds = model.predict(val_x, batch_size = 256, verbose = 1)
print(np.sum(preds, axis = 0))
print(np.sum(val_labels, axis = 0))
plt.imshow(
    confusion_matrix(
        enc.inverse_transform(preds), 
        enc.inverse_transform(val_labels), 
        normalize = "true"
    )
)

In [ ]:
log_dir = os.path.join(
    "Singlelogs",
    "fit",
    "addDense",
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),
)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
model.fit(train_x, train_labels,
          epochs = 100, 
          shuffle = True,
          validation_data = (val_x, val_labels),
          batch_size = 1024,
         class_weight = class_weights,
         callbacks=[tensorboard_callback])

In [ ]:
preds = model.predict(train_x, batch_size = 256, verbose = 1)
print(np.sum(preds, axis = 0))
print(np.sum(train_labels, axis = 0))
plt.imshow(
    confusion_matrix(
        enc.inverse_transform(preds), 
        enc.inverse_transform(train_labels), 
        normalize = "true"
    )
)

In [ ]:
preds = model.predict(val_x, batch_size = 256, verbose = 1)
print(np.sum(preds, axis = 0))
print(np.sum(val_labels, axis = 0))
plt.imshow(
    confusion_matrix(
        enc.inverse_transform(preds), 
        enc.inverse_transform(val_labels), 
        normalize = "true"
    )
)